# structural mechanics on tetrahedral spheres
Tim Tyree<br>
12.2.2020

In [1]:
#pylab
%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

import skimage as sk
from skimage import measure, filters

from numba import njit, jit, vectorize
from PIL import Image
import imageio, pyglet

#triangular meshes only
import trimesh

#tetrahedral meshes + visualization
# !pip install pyvista
# !pip install tetgen
import pyvista as pv
import tetgen

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
# width = 512
# height = 512
# channel_no = 3

# #load the libraries
from lib import *

#use cuda via numba
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8
from numba.typed import List
# from lib.contours_to_tips import *

%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


In [2]:
# !pip3 install pymeshfix

In [3]:
#in darkmode for jupyter notebooks
# !jt -t monokai -f fira -fs 13 -nf ptsans -nfs 11 -N -kl -cursw 5 -cursc r -cellw 95% -T

#Hack for images to obey darkmode
import seaborn as sns

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# import a spherical surface mesh and turn it into a spherical tetrahedral mesh

In [4]:
os.chdir(nb_dir)

input_file_name = f'../data/spherical_meshes/spherical_mesh_64.stl'
# input_file_name = f'../data/tetrahedral_meshes/tetrahedral_sphere.ply'
print(input_file_name)
mesh_trimesh = trimesh.load(input_file_name)
t = 0.
# mesh_trimesh.vertices -= mesh_trimesh.center_mass
#normalize the mean radius to 1
# mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))

vertices_trimesh = np.array(mesh_trimesh.vertices)
faces_trimesh = np.array(mesh_trimesh.faces)

# print(mesh.volume)
# mesh_trimesh.show()

assert(mesh_trimesh.is_winding_consistent)
assert(mesh_trimesh.is_watertight)
assert(mesh_trimesh.is_volume)

face_normals all zero, ignoring!


../data/spherical_meshes/spherical_mesh_64.stl


In [6]:
#Nota Bene: trimesh parses .ply metadata 
# print(mesh.metadata.keys())
# print(mesh.metadata['ply_raw'].keys())
# # vertex_length = mesh.metadata['ply_raw']['vertex']['length']
# #TODO(much later): flattening each column is necessary to cast the .ply metadata into a pandas.DataFrame
# # df = pd.DataFrame(mesh.metadata['ply_raw']['vertex']['data'], index = range(vertex_length))
# # plt.hist(mesh.metadata['ply_raw']['vertex']['data']['x'].flatten())
# #about a third of the x values here are zero.
# sum((mesh.metadata['ply_raw']['vertex']['data']['x']==0).flatten())
# #store local state information in the mesh as metadata
# mesh.metadata['ply_raw']['vertex']['data']['s'].flatten()#['t']#['x']#['nx']  #is nx the vertex normal in the x direction?
# #Nota bene: ^this will let me save properties for later.  I don't know if I can add properties or if I need to mangle the existing ones (that are probs used).
# mesh.metadata['ply_raw']['face']['data']['vertex_indices']

In [11]:
#cdb is for multi-access data (nota bene: cbd := constant data base)
tmpdir_cbd = "../data/tetrahedral_meshes/tmp/tmp.cbd"
#vtk is for preexisting visualizations
tmpdir_vtk = "../data/tetrahedral_meshes/tmp/tmp.vtk"
#png's are for rendering
tmpdir_png = "../data/tetrahedral_meshes/tmp/tmp.png"

# create a tetrahedral mesh
tet = tetgen.TetGen(vertices_trimesh,faces_trimesh)
vertices_tet, elements_tet = tet.tetrahedralize(order=1)#, mindihedral=1, minratio=1., nobisect=True, steinerleft=-1)
# vertices, elements = tet.tetrahedralize(order=1, mindihedral=20, minratio=1.5, nobisect=True, steinerleft=-1)

In [8]:
# #visualize the grid mesh
# pv.set_plot_theme('document')
# grid = tet.grid
# _,img = grid.plot(show_edges=False, window_size=None,text=f'time={t}', return_img=True,#volume=True,
#     parallel_projection=True,eye_dome_lighting=True, show_axes=False)
# plt.show()

In [8]:
#the image of the mesh is stored for later!
# type(Image.fromarray(img))
# plt.imshow(img)

# compute physical properties of the mesh

In [9]:
#TODO(eventually): make ADAVI/DAAVI - Adaptive Dissipative Asynchronous Variational Integrator for Unstructured Tetrahedral Meshes

#TODO(later): How do I control the color of tet? No. That's for later EP stuff...

#TODO(later or never): for given triangle, find the nodes that are neighbors to all three vertices and are none of the three vertices.  
# ^This is a tetrahedral element.  Sort the elements and add it to a set to prevent duplicates.  Then, iterate over all faces.  Store as tracked array, mesh.elements

#TODO(later): either make a messy function to update_mesh_vertices or select the interior vertices and make a new mesh_trimesh_current to calculate the internal volume.
# # def update_mesh_vertices(mesh, vertices_array, Nvert):
# vertices.shape
# vertices_trimesh.shape

In [10]:
t=0.
N_elements = elements.shape[0]
N_vertices = vertices.shape[0]
print(f'initialized time to {t}.')

#allocate memory for numpy arrays
# initialize nodal arrays
node_array_position = tet.node.copy()
node_array_initial_position = node_array_position.copy()
node_array_time = t*(node_array_position[:,0])
node_array_volume  = 0.*node_array_time

# initialize elemental arrays
element_array_index = tet.elem.copy()
element_array_volume  = 0.*tet.elem.copy()[:,0]
element_array_time  = t*tet.elem.copy()[:,0]

#compute the total volume of the elements (not the interior) and the barycentric nodal volumes
net_volume = 0.
affine_vec = np.array([1.,1.,1.,1.])
for K_index in range(N_elements):
    #Behold! One tetrahedral finite volume, K
    K_vertices = tet.node[tet.elem[K_index]]

    #compute unsigned volume of one element
    K_X = np.vstack([K_vertices.T,affine_vec])
    K_volume = np.abs(np.linalg.det(K_X))/6.

    net_volume += K_volume
    
    #TODO: compute element_array_volume
    element_array_volume[K_index] = K_volume
    for vid in tet.elem[K_index]:
        node_array_volume[vid] += K_volume/4.

#assert the barycentric nodal volumes sum to the same value as the net volume of elements  
assert ( np.sum(node_array_volume) == np.sum(element_array_volume) )
net_volume = np.sum(element_array_volume)

initialized time to 0.0.


In [11]:
mass_density = 1. #mass units per volume units
net_mass = net_volume*mass_density

#compute element_array_mass
element_array_mass = mass_density*element_array_volume

#compute barycentric nodal masses for the displacement invariant nodal masses
node_array_mass  = mass_density*node_array_volume

In [12]:
# initialize momentum to zero
node_array_momentum = 0.*tet.node.copy()


In [13]:
def get_D_mat(K_vertices):
    x1  = K_vertices[0,0];y1 = K_vertices[0,1];z1 = K_vertices[0,2]
    x2  = K_vertices[1,0];y2 = K_vertices[1,1];z2 = K_vertices[1,2]
    x3  = K_vertices[2,0];y3 = K_vertices[2,1];z3 = K_vertices[2,2]
    x4  = K_vertices[3,0];y4 = K_vertices[3,1];z4 = K_vertices[3,2]
    D = np.array([
              [x1-x4,x2-x4,x3-x4],
              [y1-y4,y2-y4,y3-y4],
              [z1-z4,z2-z4,z3-z4]])
    return D

def compute_D_mat(node_array_position, element_array_index, K_index):
    K_vertices = node_array_equilibrium_position[element_array_index[K_index]] 
    D = get_D_mat(K_vertices)
    return D

def compute_inverse_position(node_array_equilibrium_position, element_array_index, K_index):
    '''Example Usage: 
    X_inverse = compute_inverse_position(node_array_equilibrium_position, element_array_index, K_index)'''
    D_K = compute_D_mat(node_array_position, element_array_index, K_index)
    X_inverse = np.linalg.inv(D_K)
    #undeformed volume of element
    # W = np.linalg.det(D_K)/6.
    return X_inverse

In [14]:
#initialize strain to zero
node_array_equilibrium_position = node_array_initial_position.copy()

#precompute element_array_inverse_equilibrium_position
inverse_equilibrium_position_lst = []
for K_index in range(N_elements):
    X_inverse = compute_inverse_position(node_array_equilibrium_position, element_array_index, K_index)
    inverse_equilibrium_position_lst.append(X_inverse)
element_array_inverse_equilibrium_position = np.stack(inverse_equilibrium_position_lst,axis=0)

# #visual check showed everything was stored right
# print(element_array_inverse_equilibrium_position[0])
# print(compute_inverse_position(node_array_equilibrium_position, element_array_index, 0))

In [15]:
dict_values_system = {
    'element_array_time':node_array_time,
    'element_array_index':element_array_index,
    'element_array_mass':element_array_mass,
    'element_array_volume':element_array_volume,
    'element_array_inverse_equilibrium_position': element_array_inverse_equilibrium_position,
    'node_array_time':node_array_time,
    'node_array_position':node_array_position,
    'node_array_time':node_array_time,
    'node_array_momentum':node_array_momentum,
    'node_array_mass':node_array_mass,
    'node_array_volume':node_array_volume
}

In [16]:
#check that express all system variables are expressed as numpy arrays
print(type(node_array_time))
print(node_array_time.shape)

print(type(element_array_index))
print(element_array_index.shape)

print(type(element_array_volume))
print(element_array_volume.shape)

print(type(element_array_mass))
print(element_array_mass.shape)

print(type(element_array_inverse_equilibrium_position))
print(element_array_inverse_equilibrium_position.shape)

print(type(node_array_time))
print(node_array_time.shape)

print(type(node_array_position))
print(node_array_position.shape)

print(type(node_array_time))
print(node_array_time.shape)

print(type(node_array_momentum))
print(node_array_momentum.shape)

print(type(node_array_mass))
print(node_array_mass.shape)

print(type(node_array_volume))
print(node_array_volume.shape)

<class 'numpy.ndarray'>
(1083,)
<class 'numpy.ndarray'>
(4935, 4)
<class 'numpy.ndarray'>
(4935,)
<class 'numpy.ndarray'>
(4935,)
<class 'numpy.ndarray'>
(4935, 3, 3)
<class 'numpy.ndarray'>
(1083,)
<class 'numpy.ndarray'>
(1083, 3)
<class 'numpy.ndarray'>
(1083,)
<class 'numpy.ndarray'>
(1083, 3)
<class 'numpy.ndarray'>
(1083,)
<class 'numpy.ndarray'>
(1083,)


# compute elastic forces on each node

In [17]:
#DONE: find/derive nodal force equations for tetrahedra. 
# `sifakis-courseNotes-TheoryAndDiscretization.pdf`
# --> ctrl+F for "Algorithm 1"
#Nota Bene: ^they've got many constitutive models worked out

######################################
# Nodal Elastic Forces
######################################
def get_calc_P(mu, lam):#, one, delta):
    return get_calc_P_neohookean(mu, lam)

#TODO: njit the value returned! 
def get_calc_P_neohookean(mu, lam):
    '''returns the first Piola-Kirchoff stress tensor ( times the constant membrane thickness, delta) 
    from the Neohookean constitutive model for elastic stress.
    Example Usage - given deformation matrix F = S.dot(R):
    mu = 1.; lam = 1.; delta = 0.1; 
    calc_P = get_calc_P(mu, lam)
    P = calc_P(F)'''
    #@njit
    def calc_P(F):
        FT = np.linalg.inv(F.T)
        J = np.linalg.det(F)
        P = mu*(F-mu*FT)+lam*np.log(J)*FT
        return P
    return calc_P

In [18]:
assert ( (np.linalg.inv(F).T==np.linalg.inv(F.T)).all() ) 

NameError: name 'F' is not defined

In [ ]:
#define Lamé parameters
mu = 1; lam = 1;

X_inv = element_array_inverse_equilibrium_position[0]
zero_mat = np.zeros((4,3))
x = node_array_position
B = element_array_inverse_equilibrium_position
W = element_array_volume
calc_P = get_calc_P(mu, lam)

In [ ]:
#TODO(later): define a get_compute_nodal_elastic_forces function and return something njit'd
def compute_nodal_elastic_forces(K_vertices, K_W, Bm):
    Ds= get_D_mat(K_vertices)
    F = np.matmul(Ds,Bm)
    P = calc_P(F)
    H = -K_W*np.matmul(P,Bm.T)
    f[0] += H[0]
    f[1] += H[1]
    f[2] += H[2]
    f[3] += (-H[0] -H[1] -H[2])
    return f

def get_nodal_elastic_forces(x, f, K, B, W, element_array_index):
    Ds = compute_D_mat(node_array_position=x, element_array_index=element_array_index, K_index=K)
    Bm = B[K]
    F = np.matmul(Ds,Bm)
    P = calc_P(F)
    H = -W[K]*np.matmul(P,Bm.T)
    f[0] += H[0]
    f[1] += H[1]
    f[2] += H[2]
    f[3] += (-H[0] -H[1] -H[2])
    return f

In [ ]:
#DONE: compute the first piola-kirchoff stress tensor
#TODO: show a known test case yields a reasonable result for the nodal forces
K_index = 0
K = K_index
f = zero_mat.copy()
retval = get_nodal_elastic_forces(x, f, K, B, W, element_array_index)
print(retval)

In [ ]:
#trivial test case: translated yet undeformed initialization returns zero nodal force 
K_index = 0
K_vertices = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(K_vertices,axis=0)
K_vertices -= com
K_W = element_array_volume[K_index]
Bm  = element_array_inverse_equilibrium_position[K_index]
f = compute_nodal_elastic_forces(K_vertices, K_W, Bm)

assert(np.isclose(f,0.).all())

In [ ]:
#nontrivial test case: nodal forces when compressing in the x direction
K_index = 0
K_vertices = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(K_vertices,axis=0)
K_vertices -= com
K_vertices[:,0] *= 0.5
K_W = element_array_volume[K_index]
Bm  = element_array_inverse_equilibrium_position[K_index]
f = compute_nodal_elastic_forces(K_vertices, K_W, Bm)

# assert(np.isclose(f,0.).all())

#assert net force is zero in the x direction
assert(np.isclose(np.sum(f[:,0]),0.))

Q = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(Q,axis=0)
Q -= com
q = K_vertices

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(q[:,0],q[:,1],q[:,2], marker = 'o', color='b')
ax.scatter(Q[:,0],Q[:,1],Q[:,2], marker = 'o', color='r')

#nodal forces
ax.quiver(q[:,0],q[:,1],q[:,2], f[:,0],f[:,1],f[:,2],  length=0.1, normalize=True)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.title('nodal forces due to compression in x-direction', fontsize=10)
# plt.axis('off')
# plt.savefig(f"{nb_dir}/Figures/nodal_forces_example3.png",dpi=400)

plt.show()

In [ ]:
#nontrivial test case: nodal forces when compressing in the y direction
K_index = 0
K_vertices = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(K_vertices,axis=0)
K_vertices -= com
K_vertices[:,1] *= 0.5
K_W = element_array_volume[K_index]
Bm  = element_array_inverse_equilibrium_position[K_index]
f = compute_nodal_elastic_forces(K_vertices, K_W, Bm)

# assert(np.isclose(f,0.).all())

#assert net force is zero in the x direction
assert(np.isclose(np.sum(f[:,0]),0.))

Q = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(Q,axis=0)
Q -= com
q = K_vertices

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(q[:,0],q[:,1],q[:,2], marker = 'o', color='b')
ax.scatter(Q[:,0],Q[:,1],Q[:,2], marker = 'o', color='r')

#nodal forces
ax.quiver(q[:,0],q[:,1],q[:,2], f[:,0],f[:,1],f[:,2],  length=0.1, normalize=True)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.title('nodal forces due to compression in y-direction', fontsize=10)
# plt.axis('off')
# plt.savefig(f"{nb_dir}/Figures/nodal_forces_example3.png",dpi=400)

plt.show()

In [ ]:
#nontrivial test case: nodal forces when compressing in the z direction
K_index = 0
K_vertices = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(K_vertices,axis=0)
K_vertices -= com
K_vertices[:,2] *= 0.5
K_W = element_array_volume[K_index]
Bm  = element_array_inverse_equilibrium_position[K_index]
f = compute_nodal_elastic_forces(K_vertices, K_W, Bm)

# assert(np.isclose(f,0.).all())

#assert net force is zero in the x direction
assert(np.isclose(np.sum(f[:,0]),0.))

Q = node_array_equilibrium_position[element_array_index[K_index]]
com = np.mean(Q,axis=0)
Q -= com
q = K_vertices

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(q[:,0],q[:,1],q[:,2], marker = 'o', color='b')
ax.scatter(Q[:,0],Q[:,1],Q[:,2], marker = 'o', color='r')

#nodal forces
ax.quiver(q[:,0],q[:,1],q[:,2], f[:,0],f[:,1],f[:,2],  length=0.1, normalize=True)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.title('nodal forces due to compression in z-direction', fontsize=10)
# plt.axis('off')
# plt.savefig(f"{nb_dir}/Figures/nodal_forces_example3.png",dpi=400)

plt.show()

In [ ]:
K_vertices[:,2]

In [ ]:
#DONE: perform a trivial force calculation.  is it correct? 
#DONE: perform a nontrivial force calculation.  is it correct? 



# DONE:  collect all of this initialization functionality into lib.model.initialize.py

## DONE: collect functionality into lib.model.elastic.py

In [1]:
#TODO(later): wrap all ^this into a minimalist initialization function, something like
# Omega_0 = initialize_system(mesh, mass_array) 
# Omega_t = Omega_0.copy() #System at time t


## note that the tetrahedralized mesh is no longer a shell for a sphere.  don't worry about this now.

In [37]:
tet = tetgen.TetGen(vertices_trimesh,faces_trimesh)


In [43]:
tet.tetrahedralize?

In [36]:
lst_dist = [np.linalg.norm(n) for n in tet.node]

np.min(lst_dist)

np.max(lst_dist)

TypeError: 'NoneType' object is not iterable

In [29]:
lst_dist_raw = [np.linalg.norm(n) for n in mesh_trimesh.vertices]

In [31]:
np.min(lst_dist_raw)

0.8999999761581421

In [32]:
np.max(lst_dist_raw)

1.0000007827426811

In [60]:
# tet.mesh[np.array([1])]
# tet.f.shape
# tet.f.buffer
# tet.buffer
# tet.data
# tet.base #where the data is actually stored

AttributeError: 'TetGen' object has no attribute 'base'

In [64]:
#TODO: make tet.tetrahedralize? not make new nodes
tet.write?

In [112]:
# retval.split_bodies()
# retval.
sphere = pv.Sphere()
tet = tetgen.TetGen(sphere)
tet.tetrahedralize();
tet.grid

/usr/local/lib/python3.8/site-packages/tetgen/pytetgen.py:534: UserWarning: VTK 9 no longer accepts an offset array
  self._grid = pv.UnstructuredGrid(offset, cells, cell_type, self.node)


UnstructuredGrid,Information
N Cells,3755
N Points,6716
X Bounds,"-4.993e-01, 4.993e-01"
Y Bounds,"-4.965e-01, 4.965e-01"
Z Bounds,"-5.000e-01, 5.000e-01"
N Arrays,0


In [113]:
tet.make_manifold()

In [114]:
tet.grid

UnstructuredGrid,Information
N Cells,3755
N Points,6716
X Bounds,"-4.993e-01, 4.993e-01"
Y Bounds,"-4.965e-01, 4.965e-01"
Z Bounds,"-5.000e-01, 5.000e-01"
N Arrays,0


In [121]:
retval = tet.grid.connectivity()

In [124]:
 type(retval)

pyvista.core.pointset.UnstructuredGrid

In [127]:
rv = retval.extract_surface()

In [132]:
rv.faces.shape

(26880,)

In [133]:
rv.extrude?

In [ ]:
# # tet.write(filename=tmpdir_cbd)#, binary=False)
# tetgen.pytetgen

# retval = pv.read(input_file_name)

# retval.delaunay_3d?

# retval.texture_map_to_sphere?

# retval.surface_indices()

# retval.triangulate?



# r = retval.point_data_to_cell_data()

# tet.tetrahedralize?

### identify the elements adjacent/connected to the surface nodes and use only them for simulation
- TODO(later): Hack tetrahedralize to make it not make new nodes using `import inspect`

In [13]:
# print(tet)

In [14]:
# retval = tet.make_manifold(verbose=True)

In [15]:
# tet.elem

In [144]:
#TODO: find all elements/tetrahedra in tet that have a node on the surface
#TODO: store ^those as my elements
#TODO: go through notebook and change tet.elem to elements and tet.nodes to nodes


#TODO: sort the index number of triangles
#TODO: make a simple function that checks if a triangle exists in a tetrahedron in the tetgen mesh
#TODO: label all such 

In [16]:
# vertices_tet, elements_tet = tet.tetrahedralize(order=1)#, mindihedral=1, minratio=1., nobisect=True, steinerleft=-1)
# vertices_trimesh

In [9]:
# #TODO: make a map between nodes from mesh_trimesh to tet
# key_lst = []
# value_lst = []
# for nv, v in enumerate(vertices_trimesh):
#     for nu, u in enumerate(vertices_tet):
#         if (v==u).all():
#             key_lst.append(nv)
#             value_lst.append(nu)
# index_dict_tri_to_tet = dict(zip(key_lst,value_lst))
# print(index_dict_tri_to_tet)

apparently, index_dict_tri_to_tet turned out to be the identity map

In [17]:
N_vertices_tri = vertices_trimesh.shape[0]
N_elements_tet = elements_tet.shape[0]

In [65]:
#select all tet elements that are connected to the surface faces and/or nodes
# <--> select all tet elements that have a node_index < N_vertices_tri (at least three times)
K_index_lst_star = []
K_index_lst_surface = []
for K_index, K in enumerate(elements_tet):
    #if this element contains any surface nodes,
    if (K<N_vertices_tri).any():
        #then add it to the K_index lists
        K_index_lst_star.append(K_index)
        if sum(K<N_vertices_tri) == 3:
            K_index_lst_surface.append(K_index)
K_index_array_surface = np.array(K_index_lst_surface)
K_index_array_star = np.array(K_index_lst_star)   
elements = elements_tet[K_index_array_star]
elements_surface = elements_tet[K_index_array_surface]

In [54]:
# sphere = pv.Sphere()
# tet_s = tetgen.TetGen(sphere)
# tet_s.tetrahedralize();

In [58]:
node_indices = np.array(sorted(set(elements.flatten())))
#assert that all nodes used are indexed by range(  "number of vertices in surface elements"   )
assert ( (np.diff(node_indices)==1).all() ) 
#Nota bene: if ^this fails, then vertex reindexing is necessary

In [60]:
# "number of vertices in surface elements" 
N_vertices = node_indices.shape[0]
#now node indices is generated by range(node_indices)
vertices = vertices_tet[:N_vertices].copy()

### TODO(ignore): set the thickness of this tetrahedral spherical shell mesh to a known value
#DONE: make a boolean mask for vertices being on the surface (vid<N_??)


In [63]:
def measure_max_thickness(vertices):
    lst_dist = [np.linalg.norm(n) for n in vertices]
    return np.max(lst_dist)-np.min(lst_dist)



0.5000014600134246

In [78]:
for K_index in elements_surface:
    boo_surface = K_index<N_vertices_tri
    #assert each surface tetrahedron has exactly 1 node not on the surface
    assert ( 4-sum(boo_surface) == 1 ) 

In [81]:
#TODO: compute the outward surface normal for each surface triangle
#nota bene: this can be done using the func in mesh_trimesh.face_normals, 
# but ^that would not be usable in fast simulation code

In [146]:
def compute_unit_normal(triangle_array):
    '''triangle_array is a 3x3 numpy array.'''
    Q = triangle_array
    e1 = Q[1]-Q[0]
    e2 = Q[2]-Q[1]
    normal = np.cross(e1,e2)
    normal /= np.linalg.norm(normal)
    return normal

In [147]:
def compute_outward_unit_normal(triangle_array):
    Q = triangle_array
    normal = compute_unit_normal(triangle_array)
    #compute radial vector supposing center of mesh is at the origin and the mesh isn't horribly convex
    r = np.mean(Q,axis=0)
    #set the normal to outward
    is_outward = np.dot(normal,r)>0
    if not is_outward:
        normal *= -1
    return normal

In [155]:
#assert the normal it always outward.
for K_index,K in enumerate(elements_surface):
    fid_surface = np.sort(K)[:3]
    fid_interior = np.sort(K)[3]
    Q = vertices[fid_surface]
    normal = compute_outward_unit_normal(triangle_array=Q)
    #compute radial vector
    r = np.mean(Q,axis=0)
    
    #assert the normal is outward
    assert ( np.dot(normal,r)>0 ) 

In [ ]:
#TODO: make a modified version of ^that to do the following:
#TODO: move the new nodes to have a known thickness for each surface element


In [158]:
import Blender

ModuleNotFoundError: No module named 'Blender'

In [157]:
!BPY

/bin/bash: BPY: command not found


### scratchwerk

In [153]:
# #slowly create mapping dictionaries from elements to faces
# key_lst = []
# value_lst = []
# for nu, e in enumerate(elements_surface):
#     for nv, f in enumerate(faces_trimesh):
#         #if the surface indices match,
#         if np.isclose(np.sort(e).astype('int32')[:3],np.sort(f).astype('int32')).all():
#             key_lst.append(nu)
#             value_lst.append(nv)
# index_dict_tet_to_tri = dict(zip(key_lst,value_lst))
# index_dict_tri_to_tet = dict(zip(value_lst,key_lst))
# # print(index_dict_tri_to_tet)

In [130]:
# #check that the mappings work
# vid_lst_triangle_winding_consistent = faces_trimesh[index_dict_tet_to_tri[0]]
# print(vid_lst_triangle_winding_consistent)
# print(elements_surface[0])

[150 239 156]
[150 156 239 256]


In [154]:
# #TODO: assert the normal it always outward.
# #using the winding from elements_tri! it's consistent!
# for f_index,f in enumerate(faces_trimesh):
#     fid = index_dict_tri_to_tet[f_index]
#     eid = elements_surface[fid]
#     vid_lst_triangle_winding_consistent = faces_trimesh[fid]
#     Q = vertices_trimesh[vid_lst_triangle_winding_consistent]
#     #         normal = compute_outward_unit_normal(triangle_array=Q)
#     normal = compute_unit_normal(triangle_array=Q)
#     #compute radial vector
#     r = np.mean(Q,axis=0)
    
#     #assert the normal is outward
#     assert ( np.dot(normal,r)>0 ) 

KeyError: 87

In [152]:
#TODO: assert the normal it always outward.
#using the winding from elements_tri! it's consistent!
for K_index,K in enumerate(elements_surface):
    fid = index_dict_tri_to_tet[K_index]
    vid_lst_triangle_winding_consistent = faces_trimesh[index_dict_tet_to_tri[K_index]]
    Q = vertices_trimesh[vid_lst_triangle_winding_consistent]
    normal = compute_outward_unit_normal(triangle_array=Q)
    #compute radial vector
    r = np.mean(Q,axis=0)
    
    #assert the normal is outward
    assert ( np.dot(normal,r)>0 ) 

KeyError: 64

In [141]:
faces_trimesh[index_dict_tet_to_tri[64]]

KeyError: 64

In [50]:
from queue import PriorityQueue

q = PriorityQueue()
for z in range(1):
    q.put((4, 'Read'))
    q.put((2, 'Play'))
    q.put((5, 'Write'))
    q.put((1, 'Code'))
    q.put((3, 'Study'))



In [63]:
arr = np.array([[1,2],[1,2]])
arr @ arr
arr

array([[1, 2],
       [1, 2]])

In [65]:
while not q.empty():
    next_item = q.get()
    print(next_item)